# <center>**Chapter 10 : Building Neural Networks with Pytorch**</center>

## **PyTorch Fundamentals**
**The core data structure of pytorch is a tensor. It's a multidimensional array with a shape and data type, used for numerical computations.**

In [1]:
import torch

In [2]:
X = torch.tensor([[1.0 , 4.0 , 7.0] , [2.0 , 3.0 , 6.0]])
X

tensor([[1., 4., 7.],
        [2., 3., 6.]])

In [3]:
X.shape , X.dtype

(torch.Size([2, 3]), torch.float32)

In [6]:
# indexing in tensor works same as numpy arrays

X[0 , 1], X[: , 1]

(tensor(4.), tensor([4., 3.]))

In [7]:
10 * (X + 1.0)

tensor([[20., 50., 80.],
        [30., 40., 70.]])

**you can also convert a tensor to a numpy array using the ``numpy()``  method and create a tensor from a numpy array**

In [8]:
import numpy as np
X.numpy()

array([[1., 4., 7.],
       [2., 3., 6.]], dtype=float32)

In [9]:
torch.tensor(np.array([[1. , 4. , 7.] , [2. , 3. , 6.]]))

tensor([[1., 4., 7.],
        [2., 3., 6.]], dtype=torch.float64)

**the default precision for floats in 32 bits in pytorch whereas its 64 bits in Numpy. Its generally better to use 32 bits in deep learning because this takes half the RAM and speeds up computations , and neural networks do not actually need the extra precision offered by 64 bit floats.** 

In [10]:
# you can use ``torch.FloatTensor()`` which automatically converts the array to 32 bits

torch.FloatTensor(np.array([[1. , 4. , 7.] , [2. ,3. , 6.]]))

tensor([[1., 4., 7.],
        [2., 3., 6.]])

In [11]:
# you can also modify a tensor in place using indexing and slicing as with a numpy array

X[: , 1] = -99
X

tensor([[  1., -99.,   7.],
        [  2., -99.,   6.]])

In [14]:
# the relu method applies the ReLU activation function in place by replacing all negative values with 0s

X.relu_()
X


tensor([[1., 0., 7.],
        [2., 0., 6.]])

#### **Tip : Pytorch's inplace operations are easy to spot at a glance because their name always ends with an underscore**

## **Hardware Acceleration**

In [19]:
torch.cuda.is_available()

True

In [15]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backend.mps.is_available():
    device = "mps"
else :
    device = "cpu"

In [18]:
M = torch.tensor([[1. , 2. , 3.] , [4. , 5. , 6.]])
M = M.to(device)

In [20]:
M.device

device(type='cuda', index=0)

In [21]:
R = M @ M.T
R

tensor([[14., 32.],
        [32., 77.]], device='cuda:0')

In [23]:
M = torch.rand((1000 , 1000))  # on the CPU
%timeit M @M.T

4.47 ms ± 159 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
M = torch.rand((1000 , 1000) , device = "cuda")  # on the GPU
%timeit M @ M.T

548 μs ± 14 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## **Autograd**
**PyTorch comes with an efficient implementation of reverse mode auto differentation called autograd which stands for automatic gradients**

In [25]:
x = torch.tensor(5.0 , requires_grad=True)
f = x**2
f

tensor(25., grad_fn=<PowBackward0>)

In [30]:
f.backward
x.grad

tensor(10.)

In [32]:
learning_rate = 0.1
with torch.no_grad():
    x -= learning_rate * x.grad

In [33]:

x_detached = x.detach()
x_detached -= learning_rate * x.grad 

In [ ]:
x.grad.zero_()

### **Warning : if you forget to zero out the gradients at each training iteration , the backward() method will just accumulate them, causing incorrect gradient updates. Since there wont be any explicit error, just low performance (and perhaps infinite or NaN values) , this issue may be hard to debug**

In [35]:
# the whole traning looks like this 

learning_rate  = 0.1
x = torch.tensor(5.0 , requires_grad=True)
for iteration in range(100):
    f = x ** 2  # forward pass
    f.backward()  # backward pass
    with torch.no_grad():
        x -= learning_rate * x.grad   # gradient descent step

    x.grad.zero_() # reset the gradients

In [36]:
t = torch.tensor(2.0 , requires_grad=True)
z = t.exp() # this is an intermediate result
z += 1 # this is an inplace operation
z.backward() # runtime error

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor []], which is output 0 of ExpBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

- **Some operations such as ``exp()`` , `relu()` , `rsqrt()` , `sigmoid()` , `sqrt()` , `tan()` , and `tanh()` save their outputs in the computation graph during the forward pass , then use these outputs to compute the gradients during the backward pass.This means that you must not modify such an operation's output in place, or you will get an error during the forward pass**

- **Other operations such as `abs()` , `cos()` , `log()` , `sin()` , `square()` , and `var()` save their inputs instead of their output. Such an operation doesnot care if you modify its output in place , but you must not modify its input in place before the backward pass.**

## **Implementing Linear Regression**

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [42]:
housing = fetch_california_housing()
X_train_full , X_test , y_train_full , y_test = train_test_split(housing.data , housing.target , random_state = 42)
X_train , X_valid , y_train , y_valid = train_test_split(X_train_full , y_train_full , random_state=42)

In [44]:
X_train = torch.FloatTensor(X_train)
X_valid = torch.FloatTensor(X_valid)
X_test = torch.FloatTensor(X_test)
means = X_train.mean(dim = 0 , keepdim=True)
stds = X_train.std(dim = 0 , keepdim=True)
X_train = (X_train - means) / stds
X_valid = (X_valid - means) / stds
X_test = (X_test - means) / stds

In [45]:
# now lets convert the targerts to tensors too

y_train = torch.FloatTensor(y_train)
y_valid = torch.FloatTensor(y_valid)
y_test = torch.FloatTensor(y_test)

In [47]:
# lets create the parameters of our linear regeression model

torch.manual_seed(42)
n_features = X_train.shape[1] # there are 8 input features
w = torch.randn((n_features , 1), requires_grad = True)
b = torch.tensor(0. , requires_grad=True)

In [48]:
# we will use batch gradient descent (BGD) , using the full training set at each training step

learning_rate = 0.4
n_epochs = 20
for epoch in range(n_epochs):
    y_pred = X_train @ w + b
    loss = ((y_pred - y_train) ** 2).mean()
    loss.backward()
    with torch.no_grad():
        b -= learning_rate * b.grad
        w -= learning_rate * w.grad
        b.grad.zero_()
        w.grad.zero_()
    print(f"Epoch {epoch + 1}/{n_epochs}, Loss : {loss.item()}")

Epoch 1/20, Loss : 16.235748291015625
Epoch 2/20, Loss : 5.34673547744751
Epoch 3/20, Loss : 2.8323731422424316
Epoch 4/20, Loss : 1.944043517112732
Epoch 5/20, Loss : 1.6030147075653076
Epoch 6/20, Loss : 1.4679163694381714
Epoch 7/20, Loss : 1.4126158952713013
Epoch 8/20, Loss : 1.388764500617981
Epoch 9/20, Loss : 1.377511739730835
Epoch 10/20, Loss : 1.3714261054992676
Epoch 11/20, Loss : 1.3675490617752075
Epoch 12/20, Loss : 1.3646897077560425
Epoch 13/20, Loss : 1.3623602390289307
Epoch 14/20, Loss : 1.3603529930114746
Epoch 15/20, Loss : 1.3585747480392456
Epoch 16/20, Loss : 1.3569780588150024
Epoch 17/20, Loss : 1.3555351495742798
Epoch 18/20, Loss : 1.35422682762146
Epoch 19/20, Loss : 1.3530385494232178
Epoch 20/20, Loss : 1.35195791721344


In [49]:
X_new = X_test[:3] # pretend these are new instances
with torch.no_grad():
    y_pred = X_new @ w + b # use the trained parameters to make predictions

y_pred

tensor([[2.1702],
        [2.0141],
        [2.0942]])